In [1]:
import pandas as pd
import smtplib
from email.message import EmailMessage
import ssl

In [2]:
#Reading Data from excel file
def read_excel_data(file_path):
    try:
        df = pd.read_excel(file_path)
        return df
    except Exception as e:
        print(f"Error reading excel file: {e}")
        return None

In [3]:
file_path = 'D:\\AI\\sales_data.xlsx'
df = read_excel_data(file_path)
print(df)

        Dates     customer_name          customer_email      product  \
0  2024-03-01        Rohit Jain     rohitjain@gmail.com        Milk    
1  2024-03-02      Rahul Pandey  sirricky3108@gmail.com        Bread   
2  2024-03-03       Sarah Patel         sarah@gmail.com        Sugar   
3  2024-03-04  Xavier Rodriguez        xavier@gmail.com  Coconut Oil   
4  2024-03-05        Rohit Jain  sirricky3108@gmail.com    Dove Soap   
5  2024-03-06        Emily Chen         emily@gmail.com     Handwash   
6  2024-03-07       Jacob Smith         jacob@gmail.com   Lays Chips   
7  2024-03-08    Olivia Ramirez        olivia@gmail.com        Bread   
8  2024-03-09        Rohit Jain     rohitjain@gmail.com     Handwash   
9  2024-03-10      Rahul Pandey  sirricky3108@gmail.com  Coconut Oil   
10 2024-03-11       Sarah Patel         sarah@gmail.com        Sugar   
11 2024-03-12  Xavier Rodriguez        xavier@gmail.com    Ice Cream   
12 2024-03-13        Rohit Jain     rohitjain@gmail.com    Dove 

In [5]:
# Generating Bill of customers
def generate_bill(customer_data):
    try:
        total_amount = customer_data['quantity'] * customer_data['price']
        return total_amount.sum()
    except Exception as e:
        print(f"Error generatin Bill: {e}")
        return None

In [ ]:
if df is not None:
    #Email Configuration
    email_sender = "rydshoppingmart@gmail.com"
    email_password = "bijh gjzt rhch qosf"

    #Server Configuratiom
    smtp_server = 'smtp.gmail.com'
    smtp_port = 465

    #Sending email to unique customer
    for email in df['customer_email'].unique():
        customer_data = df[df['customer_email'] == email]
        total_amount = generate_bill(customer_data)

        #HTML and CSS content to add styles in Email
        if total_amount is not None:
            try:
                subject = "Your Monthly Bill and Purchases"
                html_content = f"""
                <head>
                        <style>
                            table {{
                                width: 100%;
                                border-collapse: collapse;
                            }}
                            th, td {{
                                border: 1px solid black;
                                padding: 8px;
                                text-align: center;
                            }}
                            th {{
                                background-color: #e6f2ff; 
                                color: Black; 
                            }}
                            td {{
                                background-color: #ffffe6;
                            }}
                        </style>
                    </head>
                    <body>
                    <br>
                    <br>
                        <div style="text-align: center;">
                        <img src="cid:logo.png" alt="Company Logo" style="width: 100%; max-width: 800px; display: block; margin: 0 auto;">
                    </div><br>
                    <br>
                    <br>
                        <h3>Dear customer, your total bill for the month is ₹{total_amount:.2f}.</h3>
                    <br>    
                        <h3>Here are the details of your purchases:</h3>
                    <br>
                        <table>
                            <tr>
                                <th><b>Date</b></th>
                                <th><b>Product</b></th>
                                <th><b>Quantity</b></th>
                                <th><b>Price</b></th>
                                <th><b>Total Amount</b></th>
                            </tr>
                """

                # Add purchase details to the HTML table
                for index, row in customer_data.iterrows():
                    html_content += f"""
                            <tr>
                                <td><b>{row['Dates'].strftime('%d-%m-%Y')}</b></td>
                                <td><b>{row['product']}</b></td>
                                <td><b>{row['quantity']}</b></td>
                                <td><b>₹{row['price']:.2f}</b></td>
                                <td><b>₹{row['Total Amount']:.2f}</b></td>
                            </tr>
                        """

                html_content += """
                        </table>
                    </body>
                    <br>
                    <br>
                    <h3><center>Thank you for shopping</centre></h3>
                    <h3><center>Do visit us again</centre></h3>
                    <br>
                    <p style="text-align:center;  font-size: 2.0em; "><a href="https://sites.google.com/view/rydshoppingmart/home">Click here to make payment</a></p>
                </html>
                """

                #Createing an email message
                em = EmailMessage()
                em['From'] = email_sender
                em['To'] = email
                em['Subject'] = subject

                #Setting HTMl text 
                em.set_content(html_content)
                em.add_alternative(html_content, subtype = "html")

                #Logo as attachment
                with open('logo.png','rb') as file:
                    logo_data = file.read()
                    em.get_payload()[0].add_related(logo_data, 'image','png', cid="logo.png")

                #SMTP configuration
                context = ssl.create_default_context()
                with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as smtp:
                    smtp.login(email_sender, email_password)
                    smtp.sendmail(email_sender, email, em.as_string())
                    

            except Exception as e:
                print(f"Error sending email to {email}: {e}")
        else:
            print("No data found in Excel")
                    